In [ ]:
# 📦 Install requirements
!pip install groq langchain faiss-cpu gradio tiktoken sentence-transformers transformers langdetect langchain-community langchain-groq

# 📚 Imports
import os
import gradio as gr
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langdetect import detect
from transformers import pipeline

# 🔐 Set Groq key
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual key

# 🧠 LLaMA 3 model via Groq
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="llama3-8b-8192")

# 🤖 Embeddings
embeddings = HuggingFaceEmbeddings()

# 🌐 Translation pipelines
translation_pipelines = {
    "en": {"to_en": lambda x: x, "from_en": lambda x: x},  # No translation
    "fr": {
        "to_en": pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en"),
        "from_en": pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")
    },
    "es": {
        "to_en": pipeline("translation", model="Helsinki-NLP/opus-mt-es-en"),
        "from_en": pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
    },
    "de": {
        "to_en": pipeline("translation", model="Helsinki-NLP/opus-mt-de-en"),
        "from_en": pipeline("translation", model="Helsinki-NLP/opus-mt-en-de")
    },
    # Add more languages if needed
}

# 📄 Load ticket data
def load_ticket_data(text):
    documents = [Document(page_content=text)]
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = splitter.split_documents(documents)
    return FAISS.from_documents(docs, embeddings)

# 🔁 QA Chain (initialized once)
qa_chain = None

# 🌐 Language detection and translation
def translate_question_to_english(question):
    lang = detect(question)
    translator = translation_pipelines.get(lang, translation_pipelines["en"])
    try:
        result = translator["to_en"](question)
        translated_text = result[0]["translation_text"] if isinstance(result, list) and isinstance(result[0], dict) else question
    except Exception as e:
        print("Translation to English failed:", e)
        translated_text = question
    return translated_text, lang

def translate_answer_to_original_lang(answer, lang):
    translator = translation_pipelines.get(lang, translation_pipelines["en"])
    try:
        result = translator["from_en"](answer)
        translated_text = result[0]["translation_text"] if isinstance(result, list) and isinstance(result[0], dict) else answer
    except Exception as e:
        print("Translation to original language failed:", e)
        translated_text = answer
    return translated_text

# 💬 Chat function
def chat_with_bot(user_input, ticket_text):
    global qa_chain

    if not ticket_text.strip():
        return "❗ Please paste some ticket history."

    try:
        if qa_chain is None:
            vectorstore = load_ticket_data(ticket_text)
            retriever = vectorstore.as_retriever()
            qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

        english_input, lang = translate_question_to_english(user_input)
        print(f"Detected language: {lang} | Translated input: {english_input}")

        english_response = qa_chain.run(english_input)
        print("LLM Response (EN):", english_response)

        translated_output = translate_answer_to_original_lang(english_response, lang)
        print("Final Output:", translated_output)

        return translated_output

    except Exception as e:
        print("Error in chat_with_bot:", e)
        return f"⚠️ Error: {str(e)}"

# 🎨 Gradio UI
with gr.Blocks() as app:
    gr.Markdown("🌍 **Multilingual Customer Support Chatbot** (Groq + HuggingFace Translation)")

    ticket_input = gr.Textbox(label="📄 Paste Customer Ticket History", lines=10, placeholder="e.g. I can't log in, the reset link isn't working.")
    user_input = gr.Textbox(label="💬 Ask your question in any language")
    output = gr.Textbox(label="🤖 Bot's Reply")

    submit_btn = gr.Button("Send")
    submit_btn.click(chat_with_bot, inputs=[user_input, ticket_input], outputs=[output])

app.launch()


<ipython-input-4-83983447>:23: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d4fbf9201695fda351.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
